In [1]:
import torch
from transformers import pipeline

In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [3]:
## Execute only if 'bert-base-multi' is not in working directory
# qa_pipeline = pipeline(
#         "question-answering",
#         model="henryk/bert-base-multilingual-cased-finetuned-polish-squad2",
#         tokenizer="henryk/bert-base-multilingual-cased-finetuned-polish-squad2"
#     )

## Saving model to file

In [4]:
## Execute only if 'bert-base-multi' is not in working directory
# torch.save(qa_pipeline, 'bert-base-multi')

In [5]:
qa_pipeline = torch.load('bert-base-multi')

In [6]:
# qa_pipeline = pipeline(
#         "question-answering",
#         model="henryk/bert-base-multilingual-cased-finetuned-polish-squad2",
#         tokenizer="henryk/bert-base-multilingual-cased-finetuned-polish-squad2"
#     )
qa_pipeline({
        'context': "Warszawa jest największym miastem w Polsce pod względem \
                   liczby ludności i powierzchni",
        'question': "Jakie jest największe miasto w Polsce?"})

{'score': 0.998785138130188, 'start': 0, 'end': 8, 'answer': 'Warszawa'}

In [7]:
def getAnswer(context, query, only_ans=True):
    result = qa_pipeline({
        'context': context,
        'question': query
    })
    if only_ans:
        return result['answer']
    else:
        return result

In [8]:
with open('sklep.txt', 'r', encoding="utf-8") as file:
    data = file.read().replace('\n', ' ')
print(data)

Stolnik był wielkim panem z rodu Horeszków, o czym opowiada Gerwazy w II księdze. Jego jedynym dzieckiem była Ewa Horeszkówna, w której zakochał się Jacek Soplica. Do zaręczyn nie doszło z powodu różnicy majątkowej i wyraźnej niechęci ojca wobec tego związku; obawiając się rekuzy Soplica nie poprosił formalnie o rękę córki, zaś Horeszko przyjął swatów kasztelana witebskiego. Stolnik został zabity przez Soplicę w zamku podczas najazdu Moskali. Od tego momentu Gerwazy przysiągł sobie zemścić się na rodzie Sopliców.  Witamy w naszym sklepie Trendylook . W naszej ofercie posiadamy wiele markowych butów i ubrań. Oferujemy najlepsze ceny i darmowe zwroty do 14-dni. Białe buty adidas. Cena 249 PLN. Dostawa gratis od 200 PLN. Czerwone botki skórzane, damskie. Cena 339 PLN. Dostawa gratis od 200 PLN. Czarne buty nike. Cena 249 PLN. Dostawa gratis od 200 PLN. Czerwone buty Reebook. Cena 249 PLN. Dostawa gratis od 200 PLN dostawa darmowa od 200 PLN. Buty kozaki eleganckie skórzane. Cena 249 PLN. 

In [27]:
with open('orders.txt', 'r', encoding="utf-8") as file:
    orders = file.read().replace('\n', ' ')
print(orders)

2023-ORD-001 - Złożone - Kurtka zimowa, czapka zimowa, rękawiczki 2023-ORD-002 - W trakcie realizacji - T-shirt z nadrukiem, jeansy, trampki 2023-ORD-003 - Anulowane - Koszula w kratę, spodnie chino, pasek skórzany 2023-ORD-004 - Zakończone - Sweter z wełny, spodnie dresowe, buty sportowe 2023-ORD-005 - W trakcie realizacji - Bluza z kapturem, legginsy, sneakersy 2023-ORD-006 - Zakończone - Marynarka, koszula biała, spodnie garniturowe 2023-ORD-007 - Złożone - Kombinezon z dżinsu, klapki, plecak 2023-ORD-008 - Status: W trakcie realizacji / Przedmioty: Kurtka skórzana, t-shirt basic, spodnie jeansowe 2023-ORD-009 - Status: Zakończone / Przedmioty: Kamizelka sportowa, buty treningowe, spodnie dresowe 2023-ORD-010 - Anulowane - Sukienka w kwiaty, sandały, torebka skórzana 2023-ORD-011 - Złożone - Kurtka puchowa, szalik, rękawiczki 2023-ORD-012 - W trakcie realizacji - Koszulka polo, spodnie dresowe, trampki 2023-ORD-013 - Anulowane - Bluzka z falbankami, spódnica mini, baleriny 2023-ORD-

In [11]:
getAnswer("ojciec Janka ma na imię Kuba, a tata Kuby nazywa się Andrzej.", "Jak nazywa się tata taty Janka?", only_ans=False)

{'score': 0.46872636675834656, 'start': 53, 'end': 60, 'answer': 'Andrzej'}

In [35]:
getAnswer(context=data, query="czerwone buty?", only_ans=False)

{'score': 0.00603965949267149, 'start': 349, 'end': 356, 'answer': 'Reebook'}

In [40]:
getAnswer(context=data, query="szukam czerwone botki", only_ans=False)

{'score': 3.833120354101993e-05,
 'start': 210,
 'end': 233,
 'answer': 'Czerwone botki skórzane'}

In [12]:
getAnswer(context=data, query="Czy posiadacie w swojej ofercie skórzane botki damskie?", only_ans=False)

{'score': 0.17885731160640717,
 'start': 728,
 'end': 760,
 'answer': 'Czerwone botki skórzane, damskie'}

In [42]:
getAnswer(context=data, query="czerwone botki cena", only_ans=False)

{'score': 0.008379863575100899, 'start': 240, 'end': 247, 'answer': '339 PLN'}

In [13]:
import openai

# load and set our key
openai.api_key = open("../key.txt", "r").read().strip("\n")

In [14]:
class ChatWithGPT:
    def __init__(self):
        self.message_log = []
        # tutaj dodac wiadomosc aby chatgpt wczul sie w role wirtualnego asystenta na stronie ecommerce

    def format_query(self, question):
        return question.replace('"', '')

    def ask_chat_gpt(self, question):
        try:
            question = self.format_query(question)
        except:
            return "Please enter valid question"
        self.message_log.append({"role": "user", "content": f"{question}"})
        try:
            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", # this is "ChatGPT" $0.002 per 1k tokens
            messages=self.message_log
            )
        except:
            return "Unfortunately Chatgpt is unavailable right now :("

        assistant_reply = completion.choices[0].message.content
        self.message_log.append({"role": "assistant", "content": f"{assistant_reply}"})

        return assistant_reply

In [29]:
test = ChatWithGPT()
query = input()
ans1 = test.ask_chat_gpt(query)

In [31]:
ans1

In [33]:
print(test.message_log)

In [15]:
session = ChatWithGPT()
query = "odpowiedz na pytanie 'Jaki jest status mojego zamówienia nr 885245?' pełnym zdaniem, gdzie odpowiedzią jest 'wysłana'."
ans = session.ask_chat_gpt(query)


# "odpowiedz na pytanie 'Jaki jest status mojego zamówienia nr 345245?' pełnym zdaniem, gdzie odpowiedzią jest 'wysłana'."
# "Answer for a question in polish: 'Jaki jest status mojego zamówienia nr 345245?' in full sentence, where answer is: 'wysłana'."

In [16]:
print(ans)

Twoje zamówienie nr 885245 zostało już wysłane.


In [17]:
query="Czy posiadacie w swojej ofercie skórzane botki damskie?"
answer = getAnswer(context=data, query=query, only_ans=True)
session = ChatWithGPT()
queryGpt = f"odpowiedz na pytanie '{query}' pełnym zdaniem, gdzie odpowiedzią jest '{answer}'."
ans = session.ask_chat_gpt(queryGpt)

In [18]:
print(ans)

Tak, posiadamy w swojej ofercie Czerwone botki skórzane, damskie.


In [19]:
print(queryGpt)

odpowiedz na pytanie Czy posiadacie w swojej ofercie skórzane botki damskie? pełnym zdaniem, gdzie odpowiedzią jest 'Czerwone botki skórzane, damskie'.


In [20]:
query="Szukam czarnych butów nike."
answer = getAnswer(context=data, query=query, only_ans=True)
session = ChatWithGPT()
queryGpt = f"odpowiedz na pytanie {query} pełnym zdaniem, gdzie odpowiedzią jest '{answer}'."
ans = session.ask_chat_gpt(queryGpt)

In [21]:
ans

'"Polecam ci szukać czarnych butów Nike, ponieważ są one świetną opcją dla osób poszukujących eleganckich i uniwersalnych butów sportowych." - Czarne buty Nike.'

In [28]:
getAnswer(context=orders, query="Jaki jest status zamówienia 2023-ORD-009?", only_ans=True)

'Zakończone'

In [34]:
query="Jaki jest status zamówienia o numerze 2023-ORD-009?"
answer = getAnswer(context=orders, query=query, only_ans=True)
session = ChatWithGPT()
queryGpt = f"Jesteś wirtualnym asystentem na stronie ecommerce. Odpowiedz w odpowiedniej formie tak jak asystent na pytanie {query} pełnym zdaniem, gdzie odpowiedzią jest '{answer}'."
ans = session.ask_chat_gpt(queryGpt)

In [35]:
ans

'Status zamówienia o numerze 2023-ORD-009 jest zakończone.'

https://platform.openai.com/docs/tutorials/web-qa-embeddings
# web scraper